In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
from scipy.spatial import distance as dist
import centroidtracker
from collections import OrderedDict

C:\Users\CastroPC\Anaconda3\envs\ML-2020\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\CastroPC\Anaconda3\envs\ML-2020\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\CastroPC\Anaconda3\envs\ML-2020\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\CastroPC\Anaconda3\envs\ML-2

In [2]:
model = tf.saved_model.load("Faster_Blick4")
model = model.signatures['serving_default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
cwd = os.getcwd()
# img_path = cwd + "\\sample_input_image.jpg"
vid_path = cwd + "\\Blick2_try.mp4"
# sample_img = cv2.imread(img_path)
classes = ["None", 'Kinder Country', 'Knoppers', 'PickUp Blue', 'PickUp Blue', 'StudentFutter', 'Twix']

In [6]:
def draw_bounding_boxes(model, image, clas):
    boxes = []
    imagenp = image
    width = imagenp.shape[1]
    height = imagenp.shape[0]
    imagenp = np.expand_dims(imagenp, axis=0)
    imagenp = tf.convert_to_tensor(imagenp)
    pred = model(imagenp)

    for x in range(pred['detection_boxes'][0].shape[0]):
        det_score = pred['detection_scores'][0][x].numpy()
        if det_score >= 0.90:
            cl = int(pred['detection_classes'][0][x].numpy())
            ymin,xmin,ymax,xmax=pred['detection_boxes'][0][x].numpy()

            left, right, top, bottom = (int(round((xmin * width))), int(round((xmax * width))),
                                          int(round((ymin * height))), int(round((ymax * height))))
            boxes.append(np.array([left, bottom, right, top]))
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 3)
            cv2.putText(image, "{}".format(clas[cl]), (left -10, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
    return image, boxes



In [7]:
ct = centroidtracker.CentroidTracker(maxDisappeared=10)
cv = cv2.VideoCapture(vid_path)
framecount = 0
counter = 0
W = int(cv.get(3))
H = int(cv.get(4))
in_roi = OrderedDict()

while cv.isOpened():
    ret, frame = cv.read()
    framecount += 1
    if ret: #and framecount >= 180:
        img, box = draw_bounding_boxes(model, frame, classes)
        lin = cv2.line(img, (150, 0), (150, 640), (0, 255, 0), 2)          
        cent = ct.update(box)
        
        for objID, cntrd in cent.items():
            text = "ID {}".format(objID)
            cv2.putText(img, text, (cntrd[0] - 10, cntrd[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(img, (cntrd[0], cntrd[1]), 4, (0, 255, 0), -1)
            if cntrd[0] <= 150:
                if int(objID) not in in_roi:
                    in_roi[objID] = True
                    counter += 1
        cnt = cv2.putText(img, "COUNTER: {}".format(counter), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.imshow("Detecting", img)
        if cv2.waitKey(20) == ord('q'):
            break
    else:
        break
print(in_roi)
cv.release()
cv2. destroyAllWindows() 


OrderedDict([(0, True), (1, True), (2, True)])


In [ ]:
img, box = draw_bounding_boxes(model, sample_img, classes)
cent = ct.update(box)
if len(cent) != 0:
    for objID, cntrd in cent.items():
        print(cntrd)
        text = "ID {}".format(objID)
        cv2.putText(img, text, (cntrd[0] - 10, cntrd[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(img, (cntrd[0], cntrd[1]), 4, (0, 255, 0), -1)
cv2.imshow("Detecting", img)
x = cv2.waitKey(0)


In [ ]:
print(in_roi[2] not in in_roi)